In [108]:
%run functions.ipynb

Ready: 2023-04-26 23:42:04


# Введение

In [290]:
def make_hse_image(df, output_file_name=None, cols_without_color=[0], dedup_cols=None, col_width=1.0, row_height=0.625, font_size=28,
                     header_color='#249FD8', row_colors=['#D2DDF1', 'w'], edge_color='w',
                     bbox=[0, 0, 1, 1], header_columns=0,
                     ax=None, show=False, **kwargs):
    rating_col = df.columns[-1]
    data = df.sort_values(rating_col, ascending=False)
    data_separeted_numbers = data.copy()
    for col in data.columns[1:]:
        data_separeted_numbers[col] = data[col].apply(make_sep)
        if col in ['Пользователи, %', 'Запросы, %', 'Запросов на пользователя,\nитого']:
            data_separeted_numbers[col] = data[col].apply(lambda x: str(x).replace('.',','))
        
    if ax is None:
        size = (np.array([6.3*4, 3.94*4]))
        fig, ax = plt.subplots(figsize=size, dpi=2**8)
        ax.axis('off')

    mpl_table = ax.table(cellText=data_separeted_numbers.values, bbox=bbox, colLabels=data.columns, colLoc='center', loc='bottom')
    mpl_table.auto_set_font_size(False)
    cells_dict = dict(six.iteritems(mpl_table._cells))
    
    #im = plt.imread(r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\2ГИС 2023\Лого\Лого_2ГИС_230401_1600.png")
    #ax.figure.figimage(im, 43, 44, zorder=1, alpha=0.5)
#
    #im = plt.imread(r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\2ГИС 2023\Лого\лого_ШТ_230401_1600.png")
    #ax.figure.figimage(im, 43, 1068, zorder=1, alpha=0.5)
    
    fpath = Path(mpl.get_data_path(), r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\2ГИС 2023\Лого\arialnarrow.ttf")
    colors_dict = colorize_column(df, cols_not_to_color=cols_without_color)
    colors_dict.update(colorize_column(data, cols_not_to_color=list(range(data.shape[1]-1))), filter_zero=True)
    colors = ([color_list_full[i] for i in range(0, len(color_list_full), len(color_list_full) % 15)] + [color_list_full[-1]]) 
    
    for k in sorted(cells_dict.keys()): #k[0] - строка, k[1] - столбец
        cell = cells_dict[k]
        cell.set_edgecolor(edge_color)
        row_index = k[0]
        column_index = k[1]
        
        # Высота и цвет заголовков
        if row_index == 0:
            rotation='horizontal' if k == (0,0) else 'vertical' 
            cell.set_text_props(color='w', rotation=rotation, ha='center')
            cell.set_facecolor(header_color)
            cell.set_height(0.05)
            
        # Высота и цвет остальных строк
        else:
            ha = 'left' if column_index ==0 else 'right'
            cell.set_facecolor(row_colors[k[0]%len(row_colors)])
            cell.set_height(0.008)
            cell.set_text_props(wrap=True, ha=ha)
        
        # Ширина столбцов
        if column_index == 0:
            cell.set_width(0.006)
        elif column_index == 6:
            cell.set_width(0.0008)
        else:
            cell.set_width(0.0016)
                        
        # Цветвоая шкала
        if row_index > 0 and column_index not in cols_without_color and row_index<16:
            cell.set_facecolor(colors.pop(0).upper())       
            
        if row_index == 16:
            cell.set_text_props(color='w')
            cell.set_facecolor(header_color) 
            if column_index == 6:
                cell.set_text_props(color=header_color)
            
        # Шрифт
        cell.set_text_props(font=fpath, fontsize=font_size)
    if output_file_name:
        fig.savefig(f'{output_file_name}', bbox_inches='tight')
    if show:
        plt.show()
    plt.close()    


In [291]:
df = pd.read_excel('files/ten_city/ratings_df.xlsx').fillna(0).astype(
    {'Рубрика':str, 
'Пользователи 2020':int,
'Пользователи 2021':int,
'Пользователи 2022':int,
'Пользователи 2023':int,
'Пользователи, итого':int,
'Пользователи, %' : float,
'Запросы 2020':int,
'Запросы 2021':int,
'Запросы 2022':int,
'Запросы 2023':int,
'Запросы, итого':int,
'Запросы, %' : float,                                                                      
})

df['Запросы, %'] = df['Запросы, %'].round(1)
df['Пользователи, %'] = df['Пользователи, %'].round(1)

df_users = df[[
    'Рубрика',
    'Пользователи 2020',
    'Пользователи 2021',
    'Пользователи 2022',
    'Пользователи 2023',
    'Пользователи, итого',
    'Пользователи, %'
]].iloc[list(range(15))+[-1]].sort_values('Пользователи, итого', ascending=False)

df_requests = df[[
    'Рубрика',
    'Запросы 2020',
    'Запросы 2021',
    'Запросы 2022',
    'Запросы 2023',
    'Запросы, итого',
    'Запросы, %'
]].iloc[list(range(15))+[-1]].sort_values('Запросы, итого', ascending=False)

#249FD8
#7FA2D3
#D2DDF1

In [ ]:
now = str(datetime.datetime.now())[:-9].replace('-','').replace(' ','_').replace(':','')
make_hse_image(df_requests, f'requests_{now}.png', show=True)

In [ ]:
now = str(datetime.datetime.now())[:-9].replace('-','').replace(' ','_').replace(':','')
make_hse_image(df_users, f'users_{now}.png', show=True)

In [299]:
def make_hse_image2(df, output_file_name=None, cols_without_color=[0], dedup_cols=None, col_width=1.0, row_height=0.625, font_size=28,
                     header_color='#249FD8', row_colors=['#D2DDF1', 'w'], edge_color='w',
                     bbox=[0, 0, 1, 1], header_columns=0,
                     ax=None, show=False, **kwargs):
    rating_col = df.columns[-1]
    data = df
    data['Население'] = data['Население'].apply(make_sep)
    if ax is None:
        size = (np.array([6.3*4, 3.94*4]))
        fig, ax = plt.subplots(figsize=size, dpi=2**8)
        ax.axis('off')

    mpl_table = ax.table(cellText=data.values, bbox=bbox, colLabels=data.columns, colLoc='center', loc='bottom')
    mpl_table.auto_set_font_size(False)
    cells_dict = dict(six.iteritems(mpl_table._cells))
    
    #im = plt.imread(r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\2ГИС 2023\Лого\Лого_2ГИС_230401_1600.png")
    #ax.figure.figimage(im, 43, 44, zorder=1, alpha=0.5)
#
    #im = plt.imread(r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\2ГИС 2023\Лого\лого_ШТ_230401_1600.png")
    #ax.figure.figimage(im, 43, 1068, zorder=1, alpha=0.5)
    
    fpath = Path(mpl.get_data_path(), r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\2ГИС 2023\Лого\arialnarrow.ttf")
    #colors_dict = colorize_column(df, cols_not_to_color=cols_without_color)
    #colors_dict.update(colorize_column(data, cols_not_to_color=list(range(data.shape[1]-1))), filter_zero=True)
    colors = ([color_list_full[i] for i in range(0, len(color_list_full), len(color_list_full) % 20)] + [color_list_full[-1]]) 
    
    for k in sorted(cells_dict.keys()): #k[0] - строка, k[1] - столбец
        cell = cells_dict[k]
        cell.set_edgecolor(edge_color)
        row_index = k[0]
        column_index = k[1]
        
        # Высота и цвет заголовков
        if row_index == 0:
            rotation='horizontal' if k == (0,0) else 'vertical' 
            cell.set_text_props(color='w', ha='center')
            cell.set_facecolor(header_color)
            cell.set_height(0.04)
            
        # Высота и цвет остальных строк
        else:
            ha = 'left' if column_index in (0,1,2) else 'right'
            cell.set_facecolor(row_colors[k[0]%len(row_colors)])
            cell.set_height(0.008)
            cell.set_text_props(wrap=True, ha=ha)
        
        # Ширина столбцов
        if column_index == 0:
            cell.set_width(0.0001)
            cell.set_text_props(ha='right')
        else:
            cell.set_width(0.0006)
            
        if row_index == 0 and column_index == 0:
            cell.set_text_props(ha='center')

        # Цветвоая шкала
        if row_index > 0 and column_index == 3:
            cell.set_facecolor(colors.pop(0).upper())       
            
        if row_index == 18:
            cell.set_text_props(color='w')
            cell.set_facecolor(header_color) 
            if column_index == 1 or column_index == 2:
                cell.set_text_props(color=header_color)
            
        # Шрифт
        cell.set_text_props(font=fpath, fontsize=font_size)
    if output_file_name:
        fig.savefig(f'{output_file_name}', bbox_inches='tight')
    if show:
        plt.show()
    plt.close()    


In [ ]:
now = str(datetime.datetime.now())[:-9].replace('-','').replace(' ','_').replace(':','')
dfn = pd.read_excel(r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\ВШЭ\Таблица_Население_230425_0030.xlsx")
make_hse_image2(dfn, f'table_{now}.png', show=True)

# Рейтинги

In [10]:
ratings_to_count = [
'Рейтинг по запросам 2020',
'Рейтинг по запросам 2021',
'Рейтинг по запросам 2022',
'Рейтинг по запросам 2023',
'Рейтинг по запросам, итого',
'Рейтинг по пользователям 2020',
'Рейтинг по пользователям 2021',
'Рейтинг по пользователям 2022',
'Рейтинг по пользователям 2023',
'Рейтинг по пользователям,\nитого',
'Рейтинг постоянного роста\nзапросов 2020-2023',
'Рейтинг постоянного роста\nзапросов 2021-2023',
'Рейтинг постоянного роста\nзапросов 2022-2023',
'Рейтинг постоянного роста\nпользователей 2020-2023',
'Рейтинг постоянного роста\nпользователей 2021-2023',
'Рейтинг постоянного роста\nпользователей 2022-2023',
'Рейтинг постоянного роста\nзапросов на пользователя 2020-2023',
'Рейтинг постоянного роста\nзапросов на пользователя 2021-2023',
'Рейтинг постоянного роста\nзапросов на пользователя 2022-2023',
'Рейтинг схожести динамики\nпользователей и запросов',
'Антирейтинг по запросам 2020',
'Антирейтинг по запросам 2021',
'Антирейтинг по запросам 2022',
'Антирейтинг по запросам 2023',
'Антирейтинг по запросам, итого',
'Антирейтинг по пользователям 2020',
'Антирейтинг по пользователям 2021',
'Антирейтинг по пользователям 2022',
'Антирейтинг по пользователям 2023',
'Антирейтинг по пользователям,\nитого',
'Рейтинг постоянного падения\nзапросов 2020-2023',
'Рейтинг постоянного падения\nзапросов 2021-2023',
'Рейтинг постоянного падения\nзапросов 2022-2023',
'Рейтинг постоянного падения\nпользователей 2020-2023',
'Рейтинг постоянного падения\nпользователей 2021-2023',
'Рейтинг постоянного падения\nпользователей 2022-2023',
'Рейтинг постоянного падения\nзапросов на пользователя 2020-2023',
'Рейтинг постоянного падения\nзапросов на пользователя 2021-2023',
'Рейтинг постоянного падения\nзапросов на пользователя 2022-2023',
'Антирейтинг схожести динамики\nпользователей и запросов',
]

In [55]:
df = pd.read_excel('files/ten_city/ratings_df.xlsx')
df = df[df['Рубрика'] != 'Итог'].fillna(0)

In [292]:
ratings_df_optimized = optimize_df(df, index=50, cut_level=400, ratings_to_count=ratings_to_count)

(400, 19)
start


100%|████████████████████████████████████████████████████████████████████████████████| 400/400 [00:24<00:00, 16.37it/s]


Вертикальная оптимизация


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:23<00:00,  2.38s/it]


,Оригинальный файл,Горизонтальная оптимизация,Вертикальная оптимизация
cnt_horizontal,,,
0.0,41,41,41
1.0,50,284,180
2.0,65,21,123
3.0,47,18,49
4.0,41,8,6
5.0,42,8,0
6.0,32,4,1
7.0,11,6,0
8.0,11,5,0


In [112]:
ratings_df_optimized = optimize_df(df, index=50, cut_level=600, optimization_level=10, ratings_to_count=ratings_to_count)

(600, 19)


100%|████████████████████████████████████████████████████████████████████████████████| 600/600 [00:09<00:00, 63.92it/s]


Вертикальная оптимизация


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:26<00:00,  2.62s/it]


,Оригинальный файл,Горизонтальная оптимизация,Вертикальная оптимизация
cnt_horizontal,,,
0.0,236,236,236
1.0,76,307,195
2.0,88,18,98
3.0,26,9,36
4.0,38,6,29
5.0,19,3,5
6.0,15,2,1
7.0,12,1,0
8.0,17,3,0


In [113]:
final_df = make_final(df, ratings_df_optimized)

final_df['Запросов на пользователя,\nитого'] = final_df['Запросов на пользователя, итого']
final_df = final_df.drop(['Запросов на пользователя, итого'],axis=1)
final_df.to_excel("files/ten_city/Мастер_таблица_2ГИС_10_городов_230426_1530.xlsx", index=False)

C:\Users\Mamed\AppData\Local\Temp\ipykernel_13128\580798988.py:386: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  final_svod['Рейтинг суммарный'] = final_svod[optimized_svod.columns].sum(axis=1)
C:\Users\Mamed\AppData\Local\Temp\ipykernel_13128\580798988.py:387: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  final_svod['Рейтинг средний'] = final_svod[optimized_svod.columns].mean(axis=1)


In [114]:
make_ratings_word("files/ten_city/Мастер_таблица_2ГИС_10_городов_230426_1530.xlsx", header_color='#249FD8', row_color='#D2DDF1', 
                  size_x=6.3*4, size_y=3.94*4, cnt=9999, font_size=28, img_w=16.5, img_h=10, blank=False, word_sample='files/report_ten_city.docx')

files/ten_city/Мастер_таблица_2ГИС_10_городов_report_230426_2349.docx


100%|██████████████████████████████████████████████████████████████████████████████████| 40/40 [01:47<00:00,  2.68s/it]


In [4]:
def make_ratings_image(df, output_file_name=None, cols_without_color=[0], dedup_cols=None, col_width=1.0, row_height=0.625, font_size=18,
                     header_color='#C0504D', row_colors=['#F2DCDB', 'w'], edge_color='w',
                     bbox=[0, 0, 1, 1], header_columns=0,
                     ax=None, size_x=17.5, size_y=10.8, **kwargs):
    rating_col = df.iloc[:,-1].name
    data = df[df[rating_col]>0].sort_values(rating_col)
    data_separeted_numbers = data.copy()
    for col in data.columns[1:]:
        data_separeted_numbers[col] = data[col].apply(make_sep)
        if col in ['Пользователи, %', 'Запросы, %', 'Запросов на пользователя,\nитого']:
            data_separeted_numbers[col] = data[col].apply(lambda x: str(x).replace('.',','))
        
    if ax is None:
        size = (np.array([size_x, size_y]))
        fig, ax = plt.subplots(figsize=size, dpi=2**8)
        ax.axis('off')

    mpl_table = ax.table(cellText=data_separeted_numbers.values, bbox=bbox, colLabels=data.columns, colLoc='center', loc='bottom')
    mpl_table.auto_set_font_size(False)
    cells_dict = dict(six.iteritems(mpl_table._cells))
    
    im = plt.imread(r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\2ГИС 2023\Лого\01_Logo_HSE_full_rus_PANTONE_for_dark_2.png")
    ax.figure.figimage(im, 85, 2800, zorder=1, alpha=1)
    
    fpath = Path(mpl.get_data_path(), r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\2ГИС 2023\Лого\arialnarrow.ttf")
    colors_dict = colorize_column(df, cols_not_to_color=cols_without_color)
    colors_dict.update(colorize_column(data, cols_not_to_color=list(range(data.shape[1]-1))), filter_zero=True)
    
    for k in sorted(cells_dict.keys()): #k[0] - строка, k[1] - столбец
        cell = cells_dict[k]
        cell.set_edgecolor(edge_color)
        row_index = k[0]
        column_index = k[1]
        
        # Высота и цвет заголовков
        if row_index == 0:
            rotation='horizontal' if k == (0,0) else 'vertical' 
            cell.set_text_props(color='w', rotation=rotation, ha='center')
            cell.set_facecolor(header_color)
            cell.set_height(0.09)
            
        # Высота и цвет остальных строк
        else:
            ha = 'left' if column_index==0 else 'right'
            cell.set_facecolor(row_colors[k[0]%len(row_colors)])
            cell.set_height(0.008)
            cell.set_text_props(wrap=True, ha=ha)
    
        # Ширина столбцов
        if column_index == 0:
            cell.set_width(0.006)
        else:
            cell.set_width(0.0012)
                        
        # Цветвоая шкала
        if row_index > 0 and column_index not in cols_without_color:
            row_index_origin = data[data.iloc[:,-1] == row_index].index[0]
            color_value = colors_dict[column_index][row_index_origin]
            cell.set_facecolor(color_value) 
        
        # Раскраска дублей
        if dedup_cols is not None:
            for i in dedup_cols:
                dups = df.iloc[:,i][(df.iloc[:,i].duplicated())&(('Рейтинг рубрики' not in df.columns)|(df['Рейтинг рубрики'] > 0))].values 
                if row_index > 0 and column_index == i and cell.get_text().get_text() in dups:
                    cell.set_facecolor('#FFC7CE')
                    cell.set_text_props(color='#9C0006')         
        # Шрифт
        cell.set_text_props(font=fpath, fontsize=font_size)
    if output_file_name:
        fig.savefig(f'{output_file_name}', bbox_inches='tight')
    #plt.show()
    plt.close()    


In [334]:
df = pd.read_excel("files/ten_city/Мастер_таблица_2ГИС_10_городов_230426_1530.xlsx")

In [335]:
df = df.rename({
    'Рейтинг постоянного падения\nзапросов на пользователя 2022-2023' : 'Рейтинг падения запросов\nна пользователя 2022-2023',
    'Рейтинг постоянного падения\nзапросов на пользователя 2021-2023' : 'Рейтинг падения запросов\nна пользователя 2021-2023',
    'Рейтинг постоянного падения\nзапросов на пользователя 2020-2023' : 'Рейтинг падения запросов\nна пользователя 2020-2023',
    'Рейтинг постоянного роста\nзапросов на пользователя 2022-2023' : 'Рейтинг роста запросов\nна пользователя 2022-2023',
    'Рейтинг постоянного роста\nзапросов на пользователя 2021-2023' : 'Рейтинг роста запросов\nна пользователя 2021-2023',
    'Рейтинг постоянного роста\nзапросов на пользователя 2020-2023' : 'Рейтинг роста запросов\nна пользователя 2020-2023',
    'Антирейтинг по пользователям 2023' : 'Антирейтинг\nпо пользователям 2023',
    'Антирейтинг по пользователям 2022' : 'Антирейтинг\nпо пользователям 2022',
    'Антирейтинг по пользователям 2021' : 'Антирейтинг\nпо пользователям 2021',
    'Антирейтинг по пользователям 2020' : 'Антирейтинг\nпо пользователям 2020',
}, axis=1)

df.to_excel("files/ten_city/Мастер_таблица_2ГИС_10_городов_230428_1845.xlsx", index=False)

In [336]:
sorted([ (len(i), i) for i in df.columns if '\n' not in i], reverse=True)

[(30, 'Антирейтинг по запросам, итого'),
 (29, 'Рейтинг по пользователям 2023'),
 (29, 'Рейтинг по пользователям 2022'),
 (29, 'Рейтинг по пользователям 2021'),
 (29, 'Рейтинг по пользователям 2020'),
 (29, 'Запросов на пользователя 2023'),
 (29, 'Запросов на пользователя 2022'),
 (29, 'Запросов на пользователя 2021'),
 (29, 'Запросов на пользователя 2020'),
 (28, 'Антирейтинг по запросам 2023'),
 (28, 'Антирейтинг по запросам 2022'),
 (28, 'Антирейтинг по запросам 2021'),
 (28, 'Антирейтинг по запросам 2020'),
 (26, 'Рейтинг по запросам, итого'),
 (24, 'Рейтинг по запросам 2023'),
 (24, 'Рейтинг по запросам 2022'),
 (24, 'Рейтинг по запросам 2021'),
 (24, 'Рейтинг по запросам 2020'),
 (19, 'Пользователи, итого'),
 (17, 'Рейтинг суммарный'),
 (17, 'Пользователи 2023'),
 (17, 'Пользователи 2022'),
 (17, 'Пользователи 2021'),
 (17, 'Пользователи 2020'),
 (16, 'В рейтингах, раз'),
 (15, 'Рейтинг средний'),
 (15, 'Пользователи, %'),
 (14, 'Запросы, итого'),
 (13, 'Вне рейтингов'),
 (12, 'З

In [325]:
sorted([(len(i),len(i.split('\n')[0]), len(i.split('\n')[1]), i, ) for i in df.columns if '\n' in i], reverse=True)

[(54, 29, 24, 'Антирейтинг схожести динамики\nпользователей и запросов'),
 (51, 27, 23, 'Рейтинг постоянного падения\nпользователей 2022-2023'),
 (51, 27, 23, 'Рейтинг постоянного падения\nпользователей 2021-2023'),
 (51, 27, 23, 'Рейтинг постоянного падения\nпользователей 2020-2023'),
 (50, 25, 24, 'Рейтинг схожести динамики\nпользователей и запросов'),
 (49, 25, 23, 'Рейтинг постоянного роста\nпользователей 2022-2023'),
 (49, 25, 23, 'Рейтинг постоянного роста\nпользователей 2021-2023'),
 (49, 25, 23, 'Рейтинг постоянного роста\nпользователей 2020-2023'),
 (49, 23, 25, 'Рейтинг падениязапросов\nна пользователя 2022-2023'),
 (49, 23, 25, 'Рейтинг падениязапросов\nна пользователя 2021-2023'),
 (49, 23, 25, 'Рейтинг падениязапросов\nна пользователя 2020-2023'),
 (48, 22, 25, 'Рейтинг роста запросов\nна пользователя 2022-2023'),
 (48, 22, 25, 'Рейтинг роста запросов\nна пользователя 2021-2023'),
 (48, 22, 25, 'Рейтинг роста запросов\nна пользователя 2020-2023'),
 (46, 27, 18, 'Рейтинг по

In [5]:
make_ratings_word("files/ten_city/Мастер_таблица_2ГИС_10_городов_230428_1845.xlsx", header_color='#249FD8', row_color='#D2DDF1', 
                  size_x=6.3*4, size_y=3.94*4, cnt=40, font_size=28, img_w=16.5, img_h=10, blank=False, word_sample='files/report_ten_city.docx')

files/ten_city/Мастер_таблица_2ГИС_10_городов_report_230428_1858.docx


100%|██████████████████████████████████████████████████████████████████████████████████| 40/40 [01:57<00:00,  2.94s/it]


# Запросы

In [61]:
make_requests_word('files/ten_city/requests_text_ten_city_230420_2154.xlsx', "", '#249FD8', '#D2DDF1', 
                   size_x=6.3*4, size_y=3.94*4, cnt=2, font_size=28, img_w=16.5, img_h=10, blank=True, word_sample='files/report_ten_city.docx')

files/ten_city/requests_text_ten_city_230420_2154_report_230425_2356.docx


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.30it/s]


# Графики

In [ ]:
rubrics = dd.read_parquet(f"files/ten_city/df_tx_grouped_filtered_ten_city")['rubricName'].unique().compute()
rubrics = [i for i in rubrics.to_dict().values()]
rubrics = pd.DataFrame(rubrics)
rubrics['Рейтинг рубрики'] = rubrics.index + 1

In [84]:
def make_linear_plots(input_filename, rubrics_df, cnt=9999, size_x=17.5, size_y=10.8, font_size=16, img_w=17.5, img_h=10.8, blank=False, word_sample='files/report.docx'):
    def zero(x,y):
        if x < 0:
            return str(0) +'%'
        return str(round(x,1)) +'%'
    
    dt_now = str(datetime.datetime.now())[2:-9].replace('-','').replace(':','').replace(' ','_')
    word_file = f'{input_filename[:-5]}_report_{dt_now}.docx'
    print(word_file)
    cols1 = ['Рубрика','Час','Количество запросов Астана','Количество запросов Бишкек','Количество запросов Москва','Количество запросов ОАЭ','Количество запросов Ташкент']
    cols2 = ['Рубрика','Дата','Количество запросов Астана','Количество запросов Бишкек','Количество запросов Москва','Количество запросов ОАЭ','Количество запросов Ташкент']
    cols3 = ['Рубрика','Неделя','Количество запросов Астана','Количество запросов Бишкек','Количество запросов Москва','Количество запросов ОАЭ','Количество запросов Ташкент']
    cols4 = ['Рубрика','День недели','Количество запросов Астана','Количество запросов Бишкек','Количество запросов Москва','Количество запросов ОАЭ','Количество запросов Ташкент']
    dff1 = pd.read_excel(input_filename, sheet_name='Часы')[cols1]
    dff2 = pd.read_excel(input_filename, sheet_name='Даты')[cols2]
    dff3 = pd.read_excel(input_filename, sheet_name='Недели')[cols3]
    dff4 = pd.read_excel(input_filename, sheet_name='Дни недели')[cols4]
    
    i = 0
    for rubric in tqdm(rubrics_df[:].iloc[:,0].values[0:cnt]):
        dff1_test = dff1[dff1['Рубрика'] == rubric].set_index(dff1.columns[1])
        dff2_test = dff2[dff2['Рубрика'] == rubric].set_index(dff2.columns[1])
        dff3_test = dff3[dff3['Рубрика'] == rubric].set_index(dff3.columns[1])
        dff4_test = dff4[dff4['Рубрика'] == rubric].set_index(dff4.columns[1])
        dff4_test.index = (['пн','вт','ср','чт','пт','сб','вс'])
        dff1_test.index = dff1_test.index.astype(str)
        i+=1
        j=0
        for d in [dff1_test, dff2_test, dff3_test, dff4_test]: 
            j+=1
            fig, ax = plt.subplots()
            ax = sns.lineplot(data = d, linewidth=3, palette=['#4F81BD', '#F79646', '#C0504D', '#9BBB59', '#264478'])
            plt.rcParams['figure.figsize']=17,10.8
            plt.rcParams['font.size'] = '16'
            plt.rcParams['figure.dpi'] = 2**8
            plt.rcParams['figure.facecolor'] = 'white'
            fpath = Path(mpl.get_data_path(), r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\2ГИС 2023\Лого\arialnarrow.ttf")
            
            
            plt.legend(['Астана', 'Бишкек', 'Москва', 'ОАЭ', 'Ташкент'], prop=fpath)
            plt.xticks(font=fpath)
            plt.yticks(font=fpath)
            plt.grid(visible=True, linewidth=0.3)
            ax.set_xlabel('')
            ax.yaxis.set_major_formatter(FuncFormatter(zero))
            ax.xaxis.set_major_locator(ticker.MultipleLocator(1)) 
            ax.yaxis.set_major_locator(ticker.LinearLocator(numticks=6))  
            title_text = f'{i:02}/{j}. {rubric}. '
            img_file_name = input_filename.replace('files/','')[:-5]
            img_file_name_final = f'pictures/{img_file_name}_{i:02}_{j}.png'
            if 'Час' == d.index.name:
                ax.set_title(f'{title_text}Запросы по часам, шт.', font=fpath)
            elif 'Дата' == d.index.name:
                ax.xaxis.set_major_locator(ticker.MultipleLocator(4)) 
                ax.set_title(f'{title_text}Запросы по датам, шт.', font=fpath)
            elif 'Неделя' == d.index.name:
                ax.set_title(f'{title_text}Запросы по неделям, шт.', font=fpath)
            else:
                ax.set_title(f'{title_text}Запросы по дням недели, шт.', font=fpath) 
            
            im = plt.imread(r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\2ГИС 2023\Лого\лого_2гис_232_354.png")
            ax.figure.figimage(im, 3250, 150, zorder=1, alpha=0.5)
          
            im = plt.imread(r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\2ГИС 2023\Лого\лого_шт_232_354.png")
            ax.figure.figimage(im, 200, 1800, zorder=1, alpha=0.5)

            ax.figure.savefig(img_file_name_final, bbox_inches='tight')
            add_image(word_file, img_file_name_final, rubric, img_w=img_w, img_h=img_h, blank=blank,word_sample=word_sample)
            #plt.show()
            plt.close()

In [87]:
make_linear_plots(f'files/ten_city/df_dates.xlsx', rubrics, 1,
                 size_x=6.3*4, size_y=3.94*4, img_w=16.5, img_h=10, blank=True, word_sample='files/report_ten_city.docx')

files/ten_city/df_dates_report_230426_0056.docx


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:04<00:00,  4.41s/it]


# Тренды 

In [3]:
df_schema = StructType([StructField('user', StringType(), True), StructField('proj', StringType(), True), StructField('rubricName', StringType(), True), StructField('rubricID', IntegerType(), True), StructField('time', StringType(), True), StructField('date', StringType(), True), StructField('org', StringType(), True), StructField('branch', StringType(), True), StructField('prod', StringType(), True), StructField('tx', StringType(), True), StructField('fl', StringType(), True), StructField('devmod', StringType(), True), StructField('lat', StringType(), True), StructField('lon', StringType(), True)])
df_list = [i for i in files_list_chain if 'Астана' in i or 'Бишкек' in i or 'Москва' in i or 'ОАЭ' in i or 'Ташкент' in i]
df_list_new = [i for i in files_list_chain if i not in df_list]

In [4]:
for file in tqdm(df_list_new):
    df = spark.read.option("header","true").csv(file, schema=df_schema)
    df.createOrReplaceTempView('df')
    city = file.split('\\')[2]
    df_new = spark.sql(f'''
    select 
    rubricName
    , "{city}" city
    , time
    , year(time) year_r
    , hour(time) hour_r
    , SUBSTR(time, 6, 5) date_r
    , case when weekofyear(time) > 9 then 1 else weekofyear(time) end week_r
    , case when extract(dayofweek from time) = 1 then 7
        else extract(dayofweek from time)-1
    end week_day_r

    from df
    ''').where('year_r in (2020,2021,2022,2023) and month(time) < 3 ')

    df_new.write.mode('append').parquet(f'files/ten_city/df_dates')

100%|███████████████████████████████████████████████████████████████████████████████| 20/20 [1:07:21<00:00, 202.06s/it]


In [6]:
spark.read.parquet(f'files/ten_city/df_dates').where('rubricName is not null').count()

906098201

In [7]:
spark.read.parquet(f'files/ten_city/df_dates').count()

1759325082

In [3]:
df_dates = spark.read.parquet(f'files/ten_city/df_dates')
df_dates.createOrReplaceTempView('df_dates')

In [4]:
for col_r in tqdm(['hour_r', 'date_r', 'week_r', 'week_day_r']):
    df_col = spark.sql(f'''       
    select 
        year_r
        , rubricName
        , city
        , {col_r}
        , count(*) cnt
    from df_dates
    group by
        year_r
        , rubricName
        , city
        , {col_r}
    ''')

    df_col.write.mode('overwrite').parquet(f'files/ten_city/df_dates_{col_r}')
    
    df_dates_col = spark.read.parquet(f'files/ten_city/df_dates_{col_r}')
    df_dates_col.createOrReplaceTempView('df_dates_col')
    df_norm = spark.sql(f'''       
    select 
        year_r
        , rubricName
        , city
        , {col_r}
        , sum(cnt) over(partition by rubricName, {col_r}) / sum(cnt) over(partition by rubricName)*100 cnt
    from df_dates_col
    ''')

    df_norm.write.mode('overwrite').parquet(f'files/ten_city/df_dates_norm_{col_r}')

100%|███████████████████████████████████████████████████████████████████████████████████| 4/4 [24:16<00:00, 364.05s/it]


In [72]:
path = f'files/ten_city/df_dates_rubrics.xlsx'
writer = pd.ExcelWriter(path, engine = 'xlsxwriter')

for col_r in tqdm(['hour_r', 'date_r', 'week_r', 'week_day_r']):
    df_piv = spark.read.parquet(f'files/ten_city/df_dates_norm_{col_r}')
    df_piv = df_piv.groupBy("rubricName").pivot(col_r).sum("cnt")
    
    for i in [i for i in df_piv.columns if i!='rubricName']:
        df_piv = df_piv.withColumnRenamed(i, 'c'+i.replace('-','_'))
        
    df_piv_mean = df_piv.toPandas().mean()
    df_piv_pd = df_piv.toPandas()
    for i in range(df_piv_pd.shape[0]):
        df_piv_pd.at[i, 'diff'] = abs(abs(df_piv_mean - df_piv_pd.iloc[i]).sum() / df_piv_mean.sum())
    df_piv_pd = df_piv_pd[['rubricName','diff']]
    df_piv_pd = df_piv_pd.loc[(~df_piv_pd.rubricName.str.contains('11', regex=False, na=False)) & (df_piv_pd.rubricName.notna()) 
                              & (~df_piv_pd.rubricName.str.contains('Сбер', regex=False, na=False)) & (~df_piv_pd.rubricName.str.contains('2ГИС', regex=False, na=False))]
    df_piv_pd['bucket'] = pd.qcut(df_piv_pd['diff'], q=5, labels=range(5))
    df_piv_pd['bucket_range'] = pd.qcut(df_piv_pd['diff'], q=5)
    df_piv_pd = df_piv_pd.sort_values(['bucket_range', 'diff'])
    df_piv_pd.to_excel(writer,index=False, sheet_name=col_r)
    
writer.close()

  0%|                                                                                            | 0/4 [00:00<?, ?it/s]C:\Users\Mamed\AppData\Local\Temp\ipykernel_11236\4224310882.py:11: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_piv_mean = df_piv.toPandas().mean()
 25%|█████████████████████                                                               | 1/4 [00:02<00:07,  2.36s/it]C:\Users\Mamed\AppData\Local\Temp\ipykernel_11236\4224310882.py:11: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_piv_mean = df_piv.toPandas().mean()
 50%|██████████████████████████████████████████                                          | 2/4 [00:11<00:12,  6.44s/

In [73]:
path = f'files/ten_city/df_dates_city.xlsx'
writer = pd.ExcelWriter(path, engine = 'xlsxwriter')

for col_r in tqdm(['hour_r', 'date_r', 'week_r', 'week_day_r']):
    df_piv = spark.read.parquet(f'files/ten_city/df_dates_norm_{col_r}')
    df_piv = df_piv.groupBy("city").pivot(col_r).sum("cnt")

    for i in [i for i in df_piv.columns if i!='city']:
        df_piv = df_piv.withColumnRenamed(i, 'c'+i.replace('-','_'))
        
    df_piv_mean = df_piv.toPandas().mean()
    df_piv_pd = df_piv.toPandas()
    for i in range(df_piv_pd.shape[0]):
        df_piv_pd.at[i, 'diff'] = abs(abs(df_piv_mean - df_piv_pd.iloc[i]).sum() / df_piv_mean.sum())
    df_piv_pd = df_piv_pd[['city','diff']]
    df_piv_pd['bucket'] = pd.qcut(df_piv_pd['diff'], q=5, labels=range(5))
    df_piv_pd['bucket_range'] = pd.qcut(df_piv_pd['diff'], q=5)
    df_piv_pd = df_piv_pd.sort_values(['bucket_range', 'diff'])
    df_piv_pd.to_excel(writer, index=False, sheet_name=col_r)
    
writer.close()

  0%|                                                                                            | 0/4 [00:00<?, ?it/s]C:\Users\Mamed\AppData\Local\Temp\ipykernel_11236\3917210488.py:11: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_piv_mean = df_piv.toPandas().mean()
 25%|█████████████████████                                                               | 1/4 [00:02<00:06,  2.26s/it]C:\Users\Mamed\AppData\Local\Temp\ipykernel_11236\3917210488.py:11: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_piv_mean = df_piv.toPandas().mean()
 50%|██████████████████████████████████████████                                          | 2/4 [00:07<00:08,  4.23s/

In [74]:
path = f'files/ten_city/df_dates_rubrics_city.xlsx'
writer = pd.ExcelWriter(path, engine = 'xlsxwriter')
for col_r in tqdm(['hour_r', 'date_r', 'week_r', 'week_day_r']):
    df_piv = spark.read.parquet(f'files/ten_city/df_dates_norm_{col_r}')
    df_piv = df_piv.groupBy(["rubricName", "city"]).pivot(col_r).sum("cnt")

    for i in [i for i in df_piv.columns if i!='city' and i!='rubricName']:
        df_piv = df_piv.withColumnRenamed(i, 'c'+i.replace('-','_'))
        
    df_piv_mean = df_piv.toPandas().mean()
    df_piv_pd = df_piv.toPandas()
    for i in range(df_piv_pd.shape[0]):
        df_piv_pd.at[i, 'diff'] = abs(abs(df_piv_mean - df_piv_pd.iloc[i]).sum() / df_piv_mean.sum())
    df_piv_pd = df_piv_pd[["rubricName", "city",'diff']]
    df_piv_pd = df_piv_pd.loc[(~df_piv_pd.rubricName.str.contains('11', regex=False, na=False)) & (df_piv_pd.rubricName.notna()) 
                          & (~df_piv_pd.rubricName.str.contains('Сбер', regex=False, na=False)) & (~df_piv_pd.rubricName.str.contains('2ГИС', regex=False, na=False))]
    df_piv_pd['bucket'] = pd.qcut(df_piv_pd['diff'], q=5, labels=range(5))
    df_piv_pd['bucket_range'] = pd.qcut(df_piv_pd['diff'], q=5)
    df_piv_pd = df_piv_pd.sort_values(['bucket_range', 'diff'])
    df_piv_pd.to_excel(writer,index=False, sheet_name=col_r)
writer.close()

  0%|                                                                                            | 0/4 [00:00<?, ?it/s]C:\Users\Mamed\AppData\Local\Temp\ipykernel_11236\4016236629.py:10: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_piv_mean = df_piv.toPandas().mean()
 25%|█████████████████████                                                               | 1/4 [00:22<01:07, 22.64s/it]C:\Users\Mamed\AppData\Local\Temp\ipykernel_11236\4016236629.py:10: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_piv_mean = df_piv.toPandas().mean()
 50%|██████████████████████████████████████████                                          | 2/4 [00:47<00:47, 23.93s/

In [201]:
dates = [r"C:\Users\Mamed\_Notebooks\2GIS_2023\files\ten_city\df_dates_city.xlsx",
r"C:\Users\Mamed\_Notebooks\2GIS_2023\files\ten_city\df_dates_rubrics.xlsx",
r"C:\Users\Mamed\_Notebooks\2GIS_2023\files\ten_city\df_dates_rubrics_city.xlsx"]

In [ ]:
i = 0

for file in dates:
    for col_r in ['hour_r', 'date_r', 'week_r', 'week_day_r']:
        xr = pd.read_excel(file, sheet_name=col_r)
        file_name = file.split('\\')[-1].replace('.xlsx','').replace('df_dates_','')
        sns_plot = sns.displot(data=xr, x="diff", kde=True, height=10).set(title=f'{file_name}_{col_r}'.replace('_r',''))
        fig = sns_plot.figure
        i+=1
        fig.savefig(f"pictures/ten_city/displot_{i}.png")

# Матрицы

In [94]:
file = r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\2ГИС 2023\10 городов\Мастер_таблица_2ГИС_10_городов_230419_1700_ext.xlsx"
city = 'ten_city'

In [91]:
plot_matrix(file, city, cnt_graphics=2, cnt_shars=12, step=1, size_x=6.3*4, size_y=3.94*4, font_size=28, img_w=16.5, img_h=10, blank=True, word_sample='files/report_ten_city.docx')

files/ten_city/ten_city_shars_230426_0106.docx


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:08<00:00,  4.22s/it]


,Запросы 2022,Запросы 2023,"Запросы, итого","Рост/Падение, %","Доля рынка, %",group
Рубрика,,,,,,
Рестораны,7769996.0,9838915.0,300.000000,1.266270,0.037746,Звёзды:
Торгово-развлекательные центры / Моллы,7748475.0,8521960.0,273.279853,1.099824,0.032693,Звёзды:
Гостиницы,6166348.0,7856070.0,188.612841,1.274023,0.030139,Звёзды:
Быстрое питание,5903936.0,7163944.0,186.169989,1.213418,0.027483,Звёзды:
Супермаркеты,5036586.0,5396175.0,121.916109,1.071395,0.020702,Собаки:
Бизнес-центры,4888856.0,5175877.0,125.830240,1.058709,0.019856,Собаки:
Бары,4133699.0,5079654.0,57.593162,1.228840,0.019487,Дети:
Торговые центры / Универсальные магазины,4334905.0,4982180.0,96.286342,1.149317,0.019113,Дети:
Аптеки,4800023.0,4736885.0,77.918592,0.986846,0.018172,Собаки:


In [95]:
plot_matrix_cities(cnt_graphics=2, size_x=6.3*4, size_y=3.94*4, font_size=28, img_w=16.5, img_h=10, blank=True, word_sample='files/report_ten_city.docx')

files/ten_city/ten_city_shars_230426_0108.docx
[########################################] | 100% Completed | 3.71 sms
[########################################] | 100% Completed | 3.78 s
[########################################] | 100% Completed | 3.89 s
[########################################] | 100% Completed | 3.97 s
[########################################] | 100% Completed | 4.07 s
[########################################] | 100% Completed | 4.14 s
[########################################] | 100% Completed | 4.25 s
[########################################] | 100% Completed | 4.31 s
[########################################] | 100% Completed | 4.42 s
[########################################] | 100% Completed | 4.45 s
[########################################] | 100% Completed | 4.48 s
[########################################] | 100% Completed | 4.51 s
[########################################] | 100% Completed | 4.54 s
[########################################] | 100% Comp

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:03<00:00,  1.94s/it]


,rubricName,Запросы 2020,Запросы 2021,Запросы 2022,Запросы 2023,"Запросы, итого","Рост/Падение, %","Доля рынка, %",group
city,,,,,,,,,
Алматы,Автовокзалы / Автостанции,280644,273340,345673,484328,300.000000,1.401116,0.002513,Звёзды:
Бишкек,Автовокзалы / Автостанции,55890,55565,78550,91577,59.257553,1.165843,0.000475,Дети:
ОАЭ,Автовокзалы / Автостанции,35322,32534,71690,91374,48.194000,1.274571,0.000474,Дети:
Новосибирск,Автовокзалы / Автостанции,113799,96703,70730,72460,75.004914,1.024459,0.000376,Дети:
Москва,Автовокзалы / Автостанции,75416,53850,56994,56580,50.797086,0.992736,0.000294,Дети:
Владивосток,Автовокзалы / Автостанции,22089,21978,23459,22996,17.533925,0.980263,0.000119,Дети:
СПБ,Автовокзалы / Автостанции,30678,9889,13780,14813,12.868898,1.074964,0.000077,Дети:
Астана,Автовокзалы / Автостанции,20385,6171,7809,9291,7.299342,1.189781,0.000048,Дети:
Калининград,Автовокзалы / Автостанции,12772,10024,7736,5617,5.659965,0.726086,0.000029,Собаки:
